In [ ]:
import intake
from shapely.geometry import Point, Polygon
import geopandas as gpd
import numpy as np
import xarray as xr

In [ ]:
import tempfile
import zipfile

In [ ]:
basin_path = "Clip/CalSimIII_AllWatersheds_20200417_WGS84.shp"

In [ ]:
def get_dataset(dataset_params):
    # Open catalog of available data sets using intake-esm package
    cat = intake.open_esm_datastore('https://cadcat.s3.amazonaws.com/cae-collection.json')
    cat_item = cat.search(
        activity_id=dataset_params["activity_id"],
        institution_id=dataset_params["institution_id"],
        table_id=dataset_params["table_id"], 
        variable_id=dataset_params["variable_id"],
        experiment_id=dataset_params["experiment_id"],
        grid_label=dataset_params["grid_label"],
        member_id=dataset_params["member_id"],
        source_id=dataset_params["source_id"],  
    )
    data_dict = cat_item.to_dataset_dict(
        xarray_open_kwargs={'consolidated': True},
        storage_options={'anon': True}
    )
    
    key = "{}.{}.{}.{}.{}.{}".format(
            dataset_params['activity_id'],
            dataset_params['institution_id'],
            dataset_params['source_id'],
            dataset_params['experiment_id'],
            dataset_params['table_id'],
            dataset_params['grid_label'],)
    
    return data_dict[key]

In [ ]:
# dataset_params = {
#     "activity_id": "LOCA2",
#     "institution_id": "UCSD",
#     "table_id": "day",
#     "variable_id": "pr",
#     "experiment_id": "ssp245",
#     "member_id": "r1i1p1f2",
#     "source_id": "CNRM-ESM2-1",
#     "grid_label": "d03",
# }

dataset_params = {
    "activity_id": "LOCA2",
    "institution_id": "UCSD",
    "table_id": "day",
    "variable_id": ["pr","tasmax","tasmin"],
    "experiment_id": "ssp370",
    "member_id": "r1i1p1f1",
    "source_id": "ACCESS-CM2",
    "grid_label": "d03",
}


ds = get_dataset(dataset_params)

In [ ]:
ds

In [ ]:
time_slice = (2015, 2100)
ds = ds.sel(time=slice(str(time_slice[0]), str(time_slice[1])))

In [ ]:
ds

In [ ]:
# https://geohackweek.github.io/nDarrays/05-aggregation/

In [ ]:
# time slice
time_slice = (2022, 2023)
ds = ds.sel(time=slice(str(time_slice[0]), str(time_slice[1])))

In [ ]:
ds.attrs['title']

In [ ]:
ds_precip = ds['pr'].resample(time="M").sum()
ds_temp = ds[['tasmin','tasmax']].resample(time="M").mean()
ds = xr.merge([ds_precip,ds_temp])

In [ ]:
ds

In [ ]:
ds = ds.resample(time="M").mean() # where M is for months

In [ ]:
data = ds.sel(time='2022-01-31')

In [ ]:
type(data)

In [ ]:
data.to_netcdf("test_all.nc")

In [ ]:
# SPATIAL MASKING

In [ ]:
def intersect(lat, lon, polygons):
    pt = Point(lon, lat)
    for i, row in polygons.iterrows():
        polygon = row.geometry
        #if pt.within(polygon):
        if polygon.contains(pt):
            return row.OBJECTID
            break
    return None

In [ ]:
def create_mask(time_slice, polygons, bbox):
    # initialize an empty array with same shape as ds
    mask = np.empty((ds.dims['lat'], ds.dims['lon']), dtype="object")

    # populate the array
    for i, x in enumerate(time_slice.coords['lon']):
        lon = x.values
        print(lon)

        for j, y in enumerate(time_slice.coords['lat']):
            lat = y.values
            
            if bbox["top"] > lat > bbox["bottom"] and bbox["left"] < lon < bbox["right"]:

                mask[j, i] = intersect(lat, lon, polygons)
                #mask[(ds.dims['lat'] - 1) - j, i] = intersect(lat, lon, polygons)
            else:
                mask[j, i] = None
                #mask[(ds.dims['lat'] - 1) - j, i] = None

    return mask

In [ ]:
data = gpd.read_file(basin_path)
polygons = data[['OBJECTID', 'geometry']]
bbox = {
    "top": 42.5,
    "bottom": 34.7,
    "left": -123.1,
    "right": -117.9,
}
mask = create_mask(ds, polygons, bbox)

In [ ]:
with open("Clip/mask2.npy", "wb") as f:
    np.save(f, mask)

In [ ]:
with open("Clip/mask.npy", "rb") as g:
   shp_mask = np.load(g, allow_pickle=True)

In [ ]:
flipped_shp_mask = np.flip(shp_mask, 1)

In [ ]:
mask = flipped_shp_mask

In [ ]:
ds.coords['mask'] = (('lat', 'lon'), mask)

In [ ]:
ds.mask

In [ ]:
%matplotlib inline

map_data = ds['pr'].sel(time='2022-01-31').where(ds.mask != None)

#map_data = ds['pr'].sel(time='2022-01-31').where(ds.mask == 166)
#map_data = ds['pr'].sel(time='2022-11-30')
map_data.plot()

In [ ]:
map_data.to_netcdf("test_all.nc")

In [ ]:
ds.coords

In [ ]:
mask_values = flipped_shp_mask[flipped_shp_mask != np.array(None)]

In [ ]:
oids = np.unique(mask_values)

In [ ]:
type(oids[10])

In [ ]:
time_slice = ds.pr[0]

In [ ]:
df = time_slice.to_dataframe()

In [ ]:
df.head()

In [ ]:
df = df.reset_index()

In [ ]:
df.head()

In [ ]:
point_data = ds[0].pr.to_dataframe()

In [ ]:
point_data.head()

In [ ]:
ds.lat.attrs

In [ ]:
ds.lon.attrs